# Demo – UBPD: Clasificador de documentos testimoniales

**Autor:** Manuel Daza Ramirez
**Proyecto:** Demo – UBPD: Clasificador de documentos testimoniales

Este notebook implementa un prototipo de clasificador de documentos testimoniales para la Unidad de Búsqueda de Personas dadas por Desaparecidas (UBPD), basado en un modelo de lenguaje (LLM) y una ontología mínima de tipos de documento, hechos, actores, periodo, territorio y ruteo interno.

## Objetivo técnico

- Cargar la ontología UBPD desde un archivo YAML.
- Construir prompts estructurados (system + user) que fuerzan al LLM a devolver un JSON con etiquetas controladas.
- Implementar funciones de preprocesamiento, llamada al modelo, parsing de JSON y validación de etiquetas.
- Exponer una función principal `classify_document(text)` y un ejemplo de uso con un testimonio de prueba.

## Estructura del notebook

1. Configuración del entorno y cliente OpenAI.
2. Preprocesamiento de texto.
3. Carga y serialización de la ontología (YAML → texto para el prompt).
4. Definición de prompts (system + user) y plantilla para clasificación.
5. Cliente LLM y función de llamada.
6. Utilidades de extracción y parsing de JSON.
7. Normalización, reglas de negocio y cálculo de prioridad.
8. Función principal `classify_document`.
9. Ejemplo de uso con un testimonio sintético.

> Nota: este notebook es un demo; en un entorno de producción se integraría con una base de datos y mecanismos adicionales de seguridad y auditoría.

In [1]:
# Configuración inicial del entorno
# Autor: Manuel Daza Ramirez

from dotenv import load_dotenv
import os

load_dotenv()  # reads .env automatically
api_key = os.getenv("OPENAI_API_KEY")

import json
import textwrap
from dataclasses import dataclass
from typing import List, Dict, Any

# Si usas un SDK específico (OpenAI, etc.), impórtalo aquí
from openai import OpenAI 
client = OpenAI()

# Configuración global
MODEL_NAME = "gpt-5.1"  


In [2]:
# Preprocesamiento simple

import re
import unicodedata

def normalize_unicode(text: str) -> str:
    return unicodedata.normalize("NFC", text)

def collapse_spaces(text: str) -> str:
    return re.sub(r"\s+", " ", text).strip()

def remove_headers_and_footers(text: str) -> str:
    # TODO: si conoces patrones de UBPD, agrégalos aquí.
    return text

def preprocess_text(text: str) -> str:
    text = normalize_unicode(text)
    text = remove_headers_and_footers(text)
    text = collapse_spaces(text)
    return text


In [4]:
# Carga de la ontología UBPD desde archivo YAML

import yaml

def load_ontology(path="../ontology_ubpd.yaml"):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

ONTOLOGY = load_ontology()

In [5]:
# Conversión de la ontología a texto para el prompt y definición de prompts

def ontology_to_prompt_text(ontology: dict) -> str:
    """
    Convierte la ontología cargada desde YAML a un bloque de texto
    que se incrusta dentro del prompt del modelo.
    """
    lines = []

    # Tipo de documento
    lines.append("1. tipo_documento:")
    for code, label in ontology["tipo_documento"].items():
        lines.append(f"   - \"{code}\": \"{label}\"")

    # Tipo de hecho
    lines.append("\n2. tipo_hecho:")
    for code, label in ontology["tipo_hecho"].items():
        lines.append(f"   - \"{code}\": \"{label}\"")

    # Territorio (si está en YAML)
    if "territorio" in ontology:
        lines.append("\n3. territorio:")
        if isinstance(ontology["territorio"], list):
            for item in ontology["territorio"]:
                lines.append(f"   - \"{item}\"")
        else:
            for k, v in ontology["territorio"].items():
                lines.append(f"   - \"{k}\": \"{v}\"")

    # Periodos
    lines.append("\n4. periodo:")
    for code, label in ontology["periodo"].items():
        lines.append(f"   - \"{code}\": \"{label}\"")

    # Actores
    lines.append("\n5. actores:")
    for code, label in ontology["actores"].items():
        lines.append(f"   - \"{code}\": \"{label}\"")

    # Ruteo
    lines.append("\n6. ruteo:")
    for code, label in ontology["ruteo"].items():
        lines.append(f"   - \"{code}\": \"{label}\"")

    return "\n".join(lines)

ONTOLOGY_PROMPT = ontology_to_prompt_text(ONTOLOGY)

In [8]:
SYSTEM_PROMPT = f"""
Eres un clasificador para la UBPD. Tu única tarea es devolver un JSON válido con etiquetas que usen EXCLUSIVAMENTE los códigos de esta ontología:

CÓDIGOS VÁLIDOS (listas cerradas):
{ONTOLOGY_PROMPT}

Reglas clave:
- tipo_documento ∈ lista tipo_documento.
- tipo_hecho ⊆ lista tipo_hecho (puede ser lista vacía).
- periodo ∈ lista periodo.
- actores ∈ lista actores (si no aparece ningún actor, usa ["ACT0"]).
- ruteo ∈ lista ruteo.
- territorio es una lista de nombres de departamentos de Colombia o ["No identificado"].
- highlights es una lista de frases textuales del documento (puede estar vacía).
- Si tipo_documento = "TD0" entonces ruteo = "RU0".

Formato JSON OBLIGATORIO (ni más ni menos campos):

{{
  "tipo_documento": "TDx",
  "tipo_hecho": ["THx", "..."],
  "territorio": ["Nombre departamento o 'No identificado'", "..."],
  "periodo": "PERx",
  "actores": ["ACTx", "..."],
  "ruteo": "RUx",
  "highlights": ["frase 1", "frase 2", ...]
}}

INSTRUCCIONES INTERNAS (no las muestres):
1) Lee el documento y decide las etiquetas adecuadas.
2) Construye mentalmente el JSON.
3) VERIFICA tú mismo:
   - ¿Todos los campos existen y solo esos?
   - ¿Todos los códigos pertenecen a las listas válidas?
   - ¿Se cumple la regla: si TD0 → RU0?
   - ¿Territorio es una lista y no está vacía (si no hay info, ["No identificado"])?
   - ¿highlights es una lista (posiblemente vacía)?
4) Si encuentras algún error, corrígelo antes de responder.
5) Responde SOLO el JSON final, sin texto adicional.
""".strip()


In [9]:
USER_TEMPLATE = """
Ejemplo 1 (entrada):
"Yo, María, cuento que en 1997, en el municipio de San Carlos, Antioquia, hombres armados de la guerrilla se llevaron a mi esposo. Nos tocó salir para Medellín."

Ejemplo 1 (salida):
{
  "tipo_documento": "TD1",
  "tipo_hecho": ["TH1","TH3"],
  "territorio": ["Antioquia"],
  "periodo": "PER2",
  "actores": ["ACT2"],
  "ruteo": "RU1",
  "highlights": [
    "1997, en el municipio de San Carlos, Antioquia",
    "se llevaron a mi esposo"
  ]
}

Ejemplo 2 (entrada):
"Oficio No. 123 de 2020. Remito informe técnico sobre la migración de datos."

Ejemplo 2 (salida):
{
  "tipo_documento": "TD0",
  "tipo_hecho": [],
  "territorio": ["No identificado"],
  "periodo": "PER5",
  "actores": ["ACT0"],
  "ruteo": "RU0",
  "highlights": []
}

Ahora clasifica este documento siguiendo estrictamente el formato de los ejemplos y las reglas del sistema:

DOCUMENTO:
{{DOCUMENTO}}

Recuerda:
- Usa únicamente los códigos válidos.
- Verifica internamente que el JSON cumple todas las reglas.
- Responde SOLO el JSON, sin comentarios adicionales.
""".strip()


def build_user_prompt(text: str) -> str:
    return USER_TEMPLATE.replace("{{DOCUMENTO}}", text.strip())

In [7]:
# Función de llamada al modelo (ejemplo genérico)

def call_llm(system_prompt: str, user_prompt: str) -> str:
    """
    Devuelve el texto bruto de la respuesta del modelo.
    Sustituye este cuerpo por la llamada real a tu proveedor.
    """
    # EJEMPLO con cliente OpenAI-style (ajusta a tu SDK real):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.0,
    )
    return response.choices[0].message.content

    raise NotImplementedError("Implementa aquí la llamada real al LLM.")


In [8]:
# Extracción segura de JSON

def extract_json_block(text: str) -> str:
    """
    Extrae el primer bloque JSON del texto de respuesta.
    Maneja casos donde el modelo pudiera envolver el JSON en texto adicional.
    """
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        raise ValueError("No se encontró un JSON válido en la respuesta del modelo.")
    return text[start:end+1]

def parse_model_response(raw_response: str) -> Dict[str, Any]:
    json_str = extract_json_block(raw_response)
    return json.loads(json_str)


In [9]:
# Helpers de validación

def fix_single_label(value: str, valid_codes, default: str) -> str:
    if value in valid_codes:
        return value
    return default

def fix_multi_labels(values, valid_codes) -> List[str]:
    if not isinstance(values, list):
        return []
    return [v for v in values if v in valid_codes]

def fix_territorio(values: List[str]) -> List[str]:
    if not isinstance(values, list) or len(values) == 0:
        return ["No identificado"]
    # Podrías normalizar mayúsculas/minúsculas aquí
    return list({v.strip() for v in values})


In [10]:
# Cálculo simple de prioridad (puedes afinarlo)

def compute_priority(pred: Dict[str, Any]) -> float:
    score = 0.0
    hechos = set(pred.get("tipo_hecho", []))
    ruteo = pred.get("ruteo")

    if "TH1" in hechos:
        score += 0.4
    if "TH4" in hechos:
        score += 0.2
    if ruteo == "RU1":
        score += 0.3
    if ruteo == "RU3":
        score += 0.1

    return min(score, 1.0)


In [11]:
# Validación completa de la predicción

def validate_and_fix(pred: Dict[str, Any]) -> Dict[str, Any]:
    pred["tipo_documento"] = fix_single_label(
        pred.get("tipo_documento"), ONTOLOGY["tipo_documento"].keys(), default="TD0"
    )

    pred["tipo_hecho"] = fix_multi_labels(
        pred.get("tipo_hecho", []), ONTOLOGY["tipo_hecho"].keys()
    )

    pred["periodo"] = fix_single_label(
        pred.get("periodo"), ONTOLOGY["periodo"].keys(), default="PER0"
    )

    pred["actores"] = fix_multi_labels(
        pred.get("actores", []), ONTOLOGY["actores"].keys()
    )

    pred["ruteo"] = fix_single_label(
        pred.get("ruteo"), ONTOLOGY["ruteo"].keys(), default="RU0"
    )

    pred["territorio"] = fix_territorio(pred.get("territorio", []))

    # Regla: si no testimonial → ruteo RU0
    if pred["tipo_documento"] == "TD0":
        pred["ruteo"] = "RU0"

    # Highlights siempre lista
    if not isinstance(pred.get("highlights"), list):
        pred["highlights"] = []

    pred["priority_score"] = compute_priority(pred)
    return pred


In [12]:
# Función principal de clasificación
# Autor: Manuel Daza Ramirez

# Función principal classify_document

def classify_document(text: str) -> Dict[str, Any]:
    clean_text = preprocess_text(text)
    user_prompt = build_user_prompt(clean_text)
    raw_response = call_llm(SYSTEM_PROMPT, user_prompt)
    raw_pred = parse_model_response(raw_response)
    final_pred = validate_and_fix(raw_pred)
    return final_pred


In [13]:
# Ejemplo de uso

testimonio = """
Yo, María, cuento que en 1997, en el municipio de San Carlos, Antioquia, hombres armados
que se identificaron como de la guerrilla se llevaron a mi esposo. Desde ese día no volvimos
a saber de él. Después de eso comenzaron las amenazas y nos tocó salir de la vereda e irnos
para Medellín, dejando todo atrás.
"""

resultado = classify_document(testimonio)
resultado


{'tipo_documento': 'TD1',
 'tipo_hecho': ['TH1', 'TH3'],
 'territorio': ['Antioquia'],
 'periodo': 'PER2',
 'actores': ['ACT2'],
 'ruteo': 'RU1',
 'highlights': ['en 1997, en el municipio de San Carlos, Antioquia',
  'hombres armados que se identificaron como de la guerrilla se llevaron a mi esposo',
  'Desde ese día no volvimos a saber de él',
  'comenzaron las amenazas y nos tocó salir de la vereda e irnos para Medellín, dejando todo atrás'],
 'priority_score': 0.7}

---
**Notebook preparado por:** Manuel Daza Ramirez  
Rol: AI Engineer (prototipo de clasificación de documentos testimoniales UBPD)  
Versión: 2025-02
